In [9]:
import scipy.io as sio

# Specify the path to your .mat file
file_path = 'C:\\Users\\UC\\Documents\\NeuMa\\22117124\\featuress.mat'

# Load the .mat file
feet = sio.loadmat(file_path)

In [ ]:
feet['']

In [11]:
Data_list = feet['data_list']
Data_array = feet['data_array']
Label_list = feet['label_list']
Label_array = feet['label_array']
Group_list = feet['groups_list']
Group_array = feet['group_array']
Features = feet['features']



In [29]:
Data_array= np.transpose(Data_array, (2, 0, 1))

Data_array.shape , Data_list.shape , Label_array.shape , Label_list.shape, Group_list.shape , Group_array. shape

((11074, 19, 300), (1, 2859), (11074, 1), (1, 2859), (1, 2859), (11074, 1))

In [32]:
from scipy import stats
def mean(data):
    return np.mean(data,axis=-1)
    
def std(data):
    return np.std(data,axis=-1)

def ptp(data):
    return np.ptp(data,axis=-1)

def var(data):
        return np.var(data,axis=-1)
def minim(data):
      return np.min(data,axis=-1)


def maxim(data):
      return np.max(data,axis=-1)

def argminim(data):
      return np.argmin(data,axis=-1)


def argmaxim(data):
      return np.argmax(data,axis=-1)

def mean_square(data):
      return np.mean(data**2,axis=-1)

def rms(data): #root mean square
      return  np.sqrt(np.mean(data**2,axis=-1))  

def abs_diffs_signal(data):
    return np.sum(np.abs(np.diff(data,axis=-1)),axis=-1)


def skewness(data):
    return stats.skew(data,axis=-1)

def kurtosis(data):
    return stats.kurtosis(data,axis=-1)

def concatenate_features(data):
    return np.concatenate((mean(data),std(data),ptp(data),var(data),minim(data),maxim(data),argminim(data),argmaxim(data),
                          mean_square(data),rms(data),abs_diffs_signal(data),
                          skewness(data),kurtosis(data)),axis=-1)

In [33]:
import numpy as np
featuress = []
for data in Data_array:
    featuress.append(concatenate_features(data))

featuress = np.array(featuress)
featuress_shape = featuress.shape
print("Features array shape:", featuress_shape)


Features array shape: (11074, 247)


In [57]:
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold, GridSearchCV
from imblearn.pipeline import Pipeline as ImbPipeline

# Define the classifier
clf = LogisticRegression()

# Define the parameter grid for GridSearchCV
param_grid = {'classifier__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4, 5, 8, 10, 12, 15]}

# Define the preprocessing steps including SMOTE
pipeline_steps = [('smote', SMOTE()), ('scaler', StandardScaler()), ('classifier', clf)]

# Create an imbalanced pipeline
imbalanced_pipeline = ImbPipeline(steps=pipeline_steps)

# Define the GroupKFold for cross-validation
gkf = GroupKFold(n_splits=5)

# Create the GridSearchCV with imbalanced pipeline
gscv = GridSearchCV(estimator=imbalanced_pipeline, param_grid=param_grid, cv=gkf, n_jobs=16)

# Fit the GridSearchCV with the features, labels, and groups
gscv.fit(featuress, Label_array, groups=Group_array)
gscv.best_score_


C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5438857587973923

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold,GridSearchCV,cross_val_score,cross_validate 
clf=LogisticRegression()
gkf=GroupKFold(n_splits=5)
param_grid = {'classifier__C': [0.01,0.05,0.1,0.5, 1,2,3,4,5,8, 10,12,15]}
pipe=Pipeline([('scaler',StandardScaler()),('classifier',clf)])
gscv=GridSearchCV(pipe,param_grid,cv=gkf,n_jobs=16)
gscv.fit(featuress,Label_array,groups=Group_array)

C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier', LogisticRegression())]),
             n_jobs=16,
             param_grid={'classifier__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4, 5,
                                           8, 10, 12, 15]})

In [36]:
gscv.best_score_


0.6088129102509987

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score

# Define base classifier (Random Forest)
base_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Instantiate Bagging Classifier
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

# Perform cross-validation
scores = cross_val_score(bagging_classifier, featuress, Label_array, cv=5)
mean_score = scores.mean()
print("Mean Accuracy (Bagging):", mean_score)


In [52]:
Data_array = np.moveaxis(Data_array,1,2)
Data_array.shape

(11074, 300, 19)

In [1]:
from tensorflow.keras.layers import Conv1D, BatchNormalization, LeakyReLU, MaxPool1D, GlobalAveragePooling1D, Dense, Dropout, AveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session

def cnnmodel():
    clear_session()
    model = Sequential()
    model.add(Conv1D(filters=5, kernel_size=3, strides=1, input_shape=(300, 19)))  # 1
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2, strides=2))  # 2
    model.add(Conv1D(filters=5, kernel_size=3, strides=1))  # 3
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2, strides=2))  # 4
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5, kernel_size=3, strides=1))  # 5
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2, strides=2))  # 6
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5, kernel_size=3, strides=1))  # 7
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2, strides=2))  # 8
    model.add(Conv1D(filters=5, kernel_size=3, strides=1))  # 9
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling1D())  # 10
    model.add(Dense(1, activation='sigmoid'))  # 11

    model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = cnnmodel()
model.summary()


NameError: name 'Input' is not defined

In [54]:
from sklearn.model_selection import GroupKFold,LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
gkf=GroupKFold()

In [55]:
accuracy=[]
for train_index, val_index in gkf.split(Data_array, Label_array, groups=Group_array):
    train_features,train_labels=Data_array[train_index],Label_array[train_index]
    val_features,val_labels=Data_array[val_index],Label_array[val_index]
    scaler=StandardScaler()
    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    model=cnnmodel()
    model.fit(train_features,train_labels,epochs=50,batch_size=128,validation_data=(val_features,val_labels))
    accuracy.append(model.evaluate(val_features,val_labels)[1])

C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.4527 - loss: 0.8637 - val_accuracy: 0.6266 - val_loss: 0.6640
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6191 - loss: 0.6691 - val_accuracy: 0.6266 - val_loss: 0.6567
Epoch 3/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6220 - loss: 0.6655 - val_accuracy: 0.6266 - val_loss: 0.6538
Epoch 4/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6248 - loss: 0.6634 - val_accuracy: 0.6266 - val_loss: 0.6530
Epoch 5/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6240 - loss: 0.6636 - val_accuracy: 0.6266 - val_loss: 0.6527
Epoch 6/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6200 - loss: 0.6652 - val_accuracy: 0.6266 - val_loss: 0.6525
Epoch 7/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6201 - loss: 0.6643 - val_accuracy: 0.6266 - val_loss: 0.6529
Epoch 8/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6185 - loss: 0.6652 - val_accuracy: 0.6266 - val_loss

C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.6109 - loss: 0.6755 - val_accuracy: 0.6411 - val_loss: 0.6484
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6223 - loss: 0.6649 - val_accuracy: 0.6411 - val_loss: 0.6469
Epoch 3/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6154 - loss: 0.6674 - val_accuracy: 0.6411 - val_loss: 0.6459
Epoch 4/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6137 - loss: 0.6676 - val_accuracy: 0.6411 - val_loss: 0.6450
Epoch 5/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6184 - loss: 0.6654 - val_accuracy: 0.6411 - val_loss: 0.6485
Epoch 6/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6083 - loss: 0.6695 - val_accuracy: 0.6411 - val_loss: 0.6460
Epoch 7/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6095 - loss: 0.6691 - val_accuracy: 0.6411 - val_loss: 0.6464
Epoch 8/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6215 - loss: 0.6626 - val_accuracy: 0.6411 - va

C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.6430 - loss: 0.6746 - val_accuracy: 0.5607 - val_loss: 0.6809
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6395 - loss: 0.6570 - val_accuracy: 0.5607 - val_loss: 0.6810
Epoch 3/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.6343 - loss: 0.6578 - val_accuracy: 0.5607 - val_loss: 0.6814
Epoch 4/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6347 - loss: 0.6572 - val_accuracy: 0.5607 - val_loss: 0.6823
Epoch 5/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6376 - loss: 0.6554 - val_accuracy: 0.5607 - val_loss: 0.6812
Epoch 6/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6373 - loss: 0.6554 - val_accuracy: 0.5607 - val_loss: 0.6812
Epoch 7/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6401 - loss: 0.6530 - val_accuracy: 0.5607 - val_loss: 0.6819
Epoch 8/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6340 - loss: 0.6562 - val_accuracy: 0.5607 - val

C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.4499 - loss: 0.7346 - val_accuracy: 0.6655 - val_loss: 0.6508
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6080 - loss: 0.6702 - val_accuracy: 0.6655 - val_loss: 0.6407
Epoch 3/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6074 - loss: 0.6700 - val_accuracy: 0.6655 - val_loss: 0.6440
Epoch 4/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.6045 - loss: 0.6713 - val_accuracy: 0.6655 - val_loss: 0.6397
Epoch 5/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5963 - loss: 0.6750 - val_accuracy: 0.6655 - val_loss: 0.6377
Epoch 6/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6037 - loss: 0.6724 - val_accuracy: 0.6655 - val_loss: 0.6414
Epoch 7/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.6142 - loss: 0.6663 - val_accuracy: 0.6655 - val_loss: 0.6425
Epoch 8/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6149 - loss: 0.6664 - val_accuracy: 0.6655 - v

C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.6250 - loss: 0.6663 - val_accuracy: 0.6111 - val_loss: 0.6634
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6212 - loss: 0.6643 - val_accuracy: 0.6111 - val_loss: 0.6640
Epoch 3/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6167 - loss: 0.6655 - val_accuracy: 0.6111 - val_loss: 0.6631
Epoch 4/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6280 - loss: 0.6610 - val_accuracy: 0.6111 - val_loss: 0.6639
Epoch 5/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6282 - loss: 0.6611 - val_accuracy: 0.6111 - val_loss: 0.6633
Epoch 6/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6219 - loss: 0.6633 - val_accuracy: 0.6111 - val_loss: 0.6619
Epoch 7/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6090 - loss: 0.6687 - val_accuracy: 0.6111 - val_loss: 0.6633
Epoch 8/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6296 - loss: 0.6581 - val_accuracy: 0.6111 - val_

In [58]:
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(Data_array, Label_array)
scaler = StandardScaler()
X_resampled = scaler.fit_transform(X_resampled.reshape(-1, X_resampled.shape[-1])).reshape(X_resampled.shape)
accuracy = []
for train_index, val_index in gkf.split(X_resampled, y_resampled, groups=Group_array):
    train_features, train_labels = X_resampled[train_index], y_resampled[train_index]
    val_features, val_labels = X_resampled[val_index], y_resampled[val_index]
    
    model = cnnmodel()
    model.fit(train_features, train_labels, epochs=50, batch_size=128, validation_data=(val_features, val_labels))
    accuracy.append(model.evaluate(val_features, val_labels)[1])


ValueError: Found array with dim 3. SMOTE expected <= 2.

In [56]:
np.mean(accuracy)


0.6177520751953125